# Context Relevance Evaluator

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://www.shutterstock.com/image-vector/concept-context-magnifying-glass-letters-600nw-2482176791.jpg"> 
</p>
</div>

## Description:

This app analyzes AI-generated responses to determine their relevance to a given context. It assigns a score, provides insights, and ensures meaningful interactions.

- Evaluates response relevance based on provided context. 

- Uses OpenAI’s models for scoring and reflection.  

- Helps maintain accuracy and coherence in AI responses.  

- Provides detailed feedback on context adherence.  

- Includes robust error handling for reliability.  



## Step 1: Environment Setup and Installation

This step installs dependencies from `requirements.txt` and checks for `OPENAI_API_KEY`.  

If installation fails, it retries up to 3 times before exiting.  

Once complete, it clears the output and prints a success message.  


In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed, retries
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


install_requirements()
clear_output()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Environment Variable Setup

This step loads environment variables from `.env` using `dotenv`.  

It checks if `OPENAI_API_KEY` is set; if missing, it exits.  

After validation, it confirms successful setup.  


In [ ]:
from dotenv import load_dotenv


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True, dotenv_path=".env")

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)

    print("Environment variables are set.")


setup_env()

## Step 3: Context Adherence Evaluation

- This code initializes the OpenAI API using an API key stored in environment variables.  

- It defines a `ContextAdherenceResult` model to structure the output, including a relevance score, a reflection on the response's alignment with the context, and any errors encountered.  

- The `detect_context_adherence` function sends a prompt and context to OpenAI, retrieves a response,  
and evaluates its adherence to the provided context.  

If an error occurs during the process, it is caught and logged, and a default response is returned.  


In [ ]:
from openai import OpenAI
import os
from pydantic import BaseModel
import traceback

openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


class ContextAdherenceResult(BaseModel):
    context_adherence_score: float
    context_adherence_reflection: str
    error: str


DEFAULT_COMPLETION_MODEL = "gpt-4o"
DEFAULT_DETECTION_MODEL = "gpt-4o"


def detect_context_adherence(prompt: str, context: str) -> ContextAdherenceResult:
    """Detects if the completion is relevant to the context

    Args:
        prompt : str : The prompt to provide the model.
        ontext : str : The context to provide the model.

    Returns:
        ContextAdherenceResult : The result of the context adherence detection.
    """
    try:
        response = openai.chat.completions.create(
            model=DEFAULT_COMPLETION_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant. Given a prompt and context, you should provide a response that is relevant to the context.",
                },
                {"role": "user", "content": f"Prompt: {prompt}\nContext: {context}"},
            ],
        )
        completion = response.choices[0].message.content

        response = openai.beta.chat.completions.parse(
            model=DEFAULT_DETECTION_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": f"""
                        Detect the 'Context Adherence' of the completion.
                        Provide a score between 0 and 1, where 0 means the completion is not relevant to the context and 1 means the completion is relevant to the context.
                        Also provide a reflection on why the completion is or is not relevant to the context.
                        Based on the provided context, provide the score based on the following criteria:
                        - The completion is relevant to the context. [Available Points: 0.2]
                        - The completion has facts or information that is related to the context. [Available Points: 0.2]
                        - The completion is coherent and makes sense in the context. [Available Points: 0.2]
                        - The completion is helpful and provides value to the context. [Available Points: 0.2]
                        - The completion does not include made-up information or unrelated content. [Available Points: 0.2]

                        The 'Available Points' are the maximum points that can be awarded for each criteria. The total score should be calculated based on the sum of the points awarded for each criteria.

                    """,
                },
                {
                    "role": "assistant",
                    "content": f"Prompt: {prompt}\nContext: {context}\nCompletion: {completion}",
                },
            ],
            response_format=ContextAdherenceResult,
        )
        return response.choices[0].message.parsed
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return ContextAdherenceResult(
            context_adherence_score=0,
            context_adherence_reflection="",
            error=str(e),
        )

## Step 4:  Running Context Adherence Check

- This code sets a prompt asking for a summary of the **Taj Mahal** and provides a context about Jimon, a software engineer who proposed to his wife at the Taj Mahal.  

- It then calls `detect_context_adherence` to check how well a generated response aligns with the given context.  

- Finally, it prints the context adherence score and a reflection on the relevance of the response.  


In [ ]:
prompt = f"Provide a summary about the Taj Mahal."

context = """
Jimon is a software engineer who has been working in the tech industry for over 10 years. He has experience in developing web applications, mobile applications, and cloud services. He is passionate about learning new technologies and enjoys working on challenging projects. In his free time, he likes to travel and explore new places. He is also an avid reader and enjoys reading books on technology and science fiction.
Jimon graduated with a degree in Computer Science from a top university and has worked at several tech companies. He is known for his problem-solving skills and attention to detail. He is a team player and enjoys collaborating with others to build innovative solutions. Jimon is currently looking for new opportunities to work on exciting projects and expand his skill set.
Jimon is married and has two children. He values work-life balance and believes in giving back to the community. He volunteers at local schools and mentors students who are interested in pursuing a career in technology. Jimon is a positive and optimistic person who believes in continuous improvement and personal growth.
Jimon proposed to his wife at Taj Mahal.
"""

result = detect_context_adherence(prompt, context)

print(f"Context Adherence Score: {result.context_adherence_score}")
print(f"Context Adherence Reflection: {result.context_adherence_reflection}")

## Conclusion:

- This app effectively evaluates context adherence by analyzing how well a generated response aligns with a given context.

- It leverages OpenAI's language models to assess relevance, coherence, and factual accuracy.

- The scoring system provides quantifiable insights into the quality of AI-generated responses.

- Error handling ensures robustness, returning a default response in case of failures.

- This tool can be useful for AI content validation, chatbots, and automated response systems.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>